In [1]:
!pip install datasets evaluate transformers[sentencepiece]
!pip install faiss-gpu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 84.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 24.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 13.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 54.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 103.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 72.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 69.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
from transformers import AutoTokenizer, AutoModel

model_ckpt = "sentence-transformers/all-mpnet-base-v2"
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
model = AutoModel.from_pretrained(model_ckpt)

In [3]:
def cls_pooling(model_output):
    return model_output.last_hidden_state[:, 0]

In [4]:
import pickle
embeddings_dataset = pickle.load(open('/content/all-mpnet-base-v2_DocumentLevel_pkl', 'rb'))
embeddings_dataset

Dataset({
    features: ['Title', 'DocID', 'URL', 'Embeddings'],
    num_rows: 200
})

In [5]:
df = embeddings_dataset.to_pandas()
df

,Title,DocID,URL,Embeddings
0,Child Development | Psychology Today,0,https://www.psychologytoday.com/us/basics/chil...,"[-0.0058288234, 0.08496683, -0.037108082, 0.12..."
1,Parenting a Child with ADHD | Psychology Today,1,https://www.psychologytoday.com/us/basics/adhd...,"[-0.0719264, -0.029214824, -0.06169279, -0.075..."
2,Raising an Adopted Child | Psychology Today,2,https://www.psychologytoday.com/us/basics/adop...,"[0.07273022, 0.056995668, 0.0070039253, -0.058..."
3,Parenting a Child with Autism | Psychology Today,3,https://www.psychologytoday.com/us/basics/auti...,"[0.0076295338, 0.0067698834, -0.041668594, 0.0..."
4,Ending Homework Battles | Psychology Today,4,https://www.psychologytoday.com/us/blog/how-ra...,"[0.003348259, -0.0010379443, 0.11785511, 0.045..."
...,...,...,...,...
195,My Kid's Toddler Tantrums Are Getting Out of H...,195,https://www.parents.com/parenting/better-paren...,"[0.11295408, 0.12687944, -0.016052825, 0.00469..."
196,10 Ways to Make Feeding Your Toddler 10 Times ...,196,https://www.parents.com/recipes/scoop-on-food/...,"[-0.13982968, 0.038722184, 0.0031211104, -0.13..."
197,Separation and Social Anxiety in Kids,197,https://www.parents.com/toddlers-preschoolers/...,"[0.03371081, 0.04600145, 0.045558274, 0.023320..."
198,How Much Does My Kid Need to Eat?,198,https://www.parents.com/toddlers-preschoolers/...,"[-0.07485011, 0.08186398, 0.10513016, 0.006583..."


In [6]:
def get_embeddings(text_list):
    encoded_input = tokenizer(
        text_list, padding=True, truncation=True, return_tensors="pt"
    )
    #encoded_input = {k: v.to(device) for k, v in encoded_input.items()}
    model_output = model(**encoded_input)
    return cls_pooling(model_output)

In [ ]:
question = ["fidgety child unable to sit still",
"child is forgetful in daily activities",
"child disregards rules",
"child not falling asleep in own bed",
"bedwetting in school age children",
"my child worries a lot",
"my child argues with me and screams",
"defiant argumentative behavior in child",
"potty training tips for toddlers",
"handling toddler tantrums",
"child is distracted and cannot focus",
"destructive and aggressive behavior in child"]

import pandas as pd

final_df = pd.DataFrame(columns=['Query','rank','Title','DocID','score','URL'])

result_dict={}


for i,question_embedding in enumerate(question):
  question_embedding = get_embeddings([question[i]]).cpu().detach().numpy()
  scores, samples = embeddings_dataset.get_nearest_examples("Embeddings", question_embedding, k=5)
  print("samples is->")
  print(samples)
  samples_df = pd.DataFrame.from_dict(samples)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  samples_df["scores"] = scores
  samples_df.sort_values("scores", ascending=False, inplace=True)
  # print("------------------------")
  # print("QUERY-> "+str(i)+" "+question[i])
  # print("------------------------")
  
  query=question[i]
  rank=1
  for _, row in samples_df.iterrows():
    title=row.Title
    docID=df.loc[df['Title'] == row.Title, 'DocID'].values[0]
    qID="q_"+str(i+1)
    dID="d_"+str(docID)
    # print(f"TITLE: {row.Title}")
    # print(f"SCORE: {row.scores}")
    # print(f"DocID: {dID}")
    # print(f"URL: {row.URL}")
    # print("=" * 50)
    # print()
    if qID not in result_dict:
      result_dict[qID]={}
      result_dict[qID][dID]=row.scores
    else:
      result_dict[qID][dID]=row.scores
    final_df.loc[len(final_df)] = [query, rank, title, docID, row.scores,row.URL]

    rank+=1
#question_embedding

In [ ]:
print(result_dict)

In [ ]:
!pip install ranx

In [11]:
from ranx import Qrels, Run
from ranx import evaluate

In [12]:
qrels_dict = { "q_1": { "d_65":2, "d_69":2, "d_173":2, "d_1":2, "d_41":2, 
                       "d_15": 1, "d_58": 1, "d_66": 1, "d_69": 1, "d_145": 1, "d_173" : 1},
"q_2": { "d_1":2,"d_41":2,"d_65":2,"d_15":1,"d_58":1,"d_66":1,"d_69":1,"d_145":1,"d_173":1},
"q_3": {"d_6":2,"d_17":2,"d_80":2,"d_120":2,"d_122":2,"d_178":2,
      "d_10":1,"d_13":1,"d_14":1,"d_16":1,"d_18":1,"d_19":1,"d_25":1,"d_79":1,"d_86":1,"d_182":1},
"q_4":{"d_32":2,"d_157":2,"d_159":2,"d_162":2,"d_187":2,
      "d_188":1,"d_33":1,"d_34":1,"d_37":1,"d_52":1,"d_62":1,"d_76":1,"d_109":1,"d_163":1,"d_164":1,"d_189":1},
"q_5":{"d_23":2,"d_26":2,"d_71":2,"d_98":2,"d_183":2,"d_186":2,
      "d_22":1,"d_27":1,"d_29":1,"d_49":1,"d_60":1,"d_63":1},
"q_6":{"d_5":2,"d_8":2,"d_9":2,"d_35":2,"d_50":2,"d_52":2,"d_53":2,"d_71":2,"d_90":2,"d_125":2,"d_153":2,"d_192":2,"d_193":2,"d_194":2,
      "d_34":1,"d_45":1,"d_138":1,"d_162":1,"d_168":1},
"q_7":{"d_178":2,"d_4":2,"d_10":2,"d_14":2,"d_80":2,"d_156":2,"d_195":2,
       "d_16":1,"d_19":1,"d_25":1,"d_36":1,"d_48":1,"d_70":1,"d_86":1,"d_91":1,"d_120":1,"d_152":1,"d_167":1,"d_175":1,"d_178":1,"d_179":1
      },
"q_8":{"d_4":2,"d_6":2,"d_13":2,"d_17":2,"d_19":2,"d_178":2,
      "d_18":1,"d_36":1,"d_70":1,"d_79":1},
"q_9":{"d_22":2,"d_27":2,"d_29":2,"d_49":2,"d_60":2,"d_63":2,"d_75":2,"d_84":2,"d_99":2,"d_100":2,"d_101":2,"d_102":2,"d_103":2,"d_104":2,"d_105":2,"d_149":2,"d_150":2,
      "d_23":1,"d_26":1,"d_46":1,"d_61":1,"d_71":1,"d_88":1,"d_89":1,"d_98":1},
"q_10":{"d_91":2,"d_10":2,"d_14":2,"d_195":2,"d_36":2,"d_48":2,"d_140":2,"d_152":2,"d_154":2,"d_156":2,"d_167":2,"d_179":2,"d_195":2,
        "d_16":1,"d_18":1,"d_25":1,"d_42":1,"d_70":1,"d_95":1,"d_175":1},
"q_11":{"d_41":2,"d_65":2,"d_169":2,
      "d_5":1,"d_15":1,"d_24":1,"d_50":1,"d_54":1,"d_58":1,"d_66":1,"d_69":1,"d_145":1,"d_173":1},
"q_12":{"d_70":2,"d_85":2,"d_174":2,"d_177":2,"d_42":2,"d_96":2,
      "d_10":1,"d_16":1,"d_18":1,"d_25":1}      
 }


run_dict = result_dict


qrels = Qrels(qrels_dict)
run = Run(run_dict)

In [13]:
# Compute score for a single metric
# evaluate(qrels, run, "ndcg@5")
final_result=evaluate(qrels, run, ["map", "ndcg"])

# Compute scores for multiple metrics at once
# evaluate(qrels, run, ["map@5", "mrr"])
print(final_result)

{'map': 0.12928437546858598, 'ndcg': 0.25413421955153764}
